In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
df = pd.read_csv('data/diabetic_data.csv', encoding = 'latin1',low_memory=False)

In [3]:
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [4]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('data/diabetic_data.csv', encoding = 'latin1',low_memory=False)
df.readmitted[df.readmitted == 'NO' ] = 0
df.readmitted[df.readmitted == '<30' ] = 1
df = df.drop(df[df.readmitted == '>30'].index)
df.drop(['encounter_id', 'patient_nbr', 'weight', 'payer_code', 'medical_specialty'], axis=1, inplace=True)
df.dropna(axis=1, how='all')
df_train, df_test = train_test_split(df, test_size=0.2, stratify=df['readmitted'])
#X = pd.get_dummies(X)

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
numeric_headers = ["time_in_hospital", "num_lab_procedures", "num_procedures", "num_medications", "number_outpatient", "number_emergency", "number_inpatient", "number_diagnoses"]
categorical_headers = ['race',
 'gender',
 'age',
 'max_glu_serum',
 'A1Cresult',
 'metformin',
 'repaglinide',
 'nateglinide',
 'chlorpropamide',
 'glimepiride',
 'acetohexamide',
 'glipizide',
 'glyburide',
 'tolbutamide',
 'pioglitazone',
 'rosiglitazone',
 'acarbose',
 'miglitol',
 'troglitazone',
 'tolazamide',
 'examide',
 'citoglipton',
 'insulin',
 'glyburide-metformin',
 'glipizide-metformin',
 'glimepiride-pioglitazone',
 'metformin-rosiglitazone',
 'metformin-pioglitazone',
 'change',
 'diabetesMed']

df_train.replace(to_replace=' ?',value=np.nan, inplace=True)
df_train.dropna(inplace=True)
df_train.reset_index()

df_test.replace(to_replace=' ?',value=np.nan, inplace=True)
df_test.dropna(inplace=True)
df_test.reset_index()

encoders = dict()

int_categorical_headers = [
'admission_type_id',
'discharge_disposition_id',
'admission_source_id',
'diag_1',
'diag_2',
'diag_3',
]

for col in categorical_headers:
    df[col] = df[col].str.strip()
    df_train[col] = df_train[col].str.strip()
    df_test[col] = df_test[col].str.strip()
    

    encoders[col] = LabelEncoder()
    df[col+'_int'] = encoders[col].fit_transform(df[col])
    df_train[col+'_int'] = encoders[col].transform(df_train[col])
    df_test[col+'_int'] = encoders[col].transform(df_test[col])
    
    
    
for col in int_categorical_headers:
    df[col+'_int'] = df[col]
    df_train[col+'_int'] = df_train[col]
    df_test[col+'_int'] = df_test[col]

for col in numeric_headers:
    df_train[col] = df_train[col].astype(np.float)
    df_test[col] = df_test[col].astype(np.float)
    df[col] = df[col].astype(np.float)
    ss = StandardScaler()
    df[col] = ss.fit_transform(df[col].values.reshape(-1, 1))
    df_train[col] = ss.transform(df_train[col].values.reshape(-1, 1))
    df_test[col] = ss.transform(df_test[col].values.reshape(-1, 1))

    
df_train.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,metformin-rosiglitazone_int,metformin-pioglitazone_int,change_int,diabetesMed_int,admission_type_id_int,discharge_disposition_id_int,admission_source_id_int,diag_1_int,diag_2_int,diag_3_int
80816,Caucasian,Female,[70-80),1,1,7,-0.114867,0.573128,-0.225250,-0.343702,...,0,0,0,1,1,1,7,518,518,401
52582,Caucasian,Male,[60-70),1,3,7,1.226448,0.725258,0.355127,-0.343702,...,0,0,1,0,1,3,7,722,324,571
93135,Asian,Female,[70-80),1,3,7,-0.450195,-1.252430,-0.805627,-0.701502,...,0,0,0,1,1,3,7,8,V09,250
31765,Caucasian,Female,[60-70),1,1,6,-1.120853,0.370288,-0.805627,-1.178569,...,0,0,1,0,1,1,6,786,427,250.6
69163,Caucasian,Female,[40-50),3,1,1,-0.785524,-1.708820,-0.225250,0.371899,...,0,0,1,1,3,1,1,241,245,250


In [6]:
from sklearn import metrics as mt

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Input
from keras.layers import Embedding, Flatten, Merge, concatenate
from keras.models import Model

In [8]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4653269956721579264
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6789848269
locality {
  bus_id: 1
}
incarnation: 1072030305897702914
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


## Starting work on the actual deep and wide network

In [9]:
categorical_headers_ints = [x+'_int' for x in categorical_headers]
df_num =  df[numeric_headers].values
X_train_num =  df_train[numeric_headers].values
X_test_num =  df_test[numeric_headers].values
y_train = df_train['readmitted'].values.astype(np.int)
y_test = df_test['readmitted'].values.astype(np.int)

In [9]:
# we need to create separate sequential models for each embedding
embed_branches = []
X_ints_train = []
X_ints_test = []
all_inputs = []
all_branch_outputs = []
X_train_num =  df_train[numeric_headers].values
X_test_num =  df_test[numeric_headers].values

for col in categorical_headers_ints:
    # encode as ints for the embedding
    X_ints_train.append( df_train[col].values )
    X_ints_test.append( df_test[col].values )

    
    # get the number of categories
    N = max(X_ints_train[-1]+1) # same as the max(df_train[col])
    
    # create embedding branch from the number of categories
    inputs = Input(shape=(1,),dtype='int32')
    all_inputs.append(inputs)
    x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
    x = Flatten()(x)
    all_branch_outputs.append(x)

# also get a dense branch of the numeric features
all_inputs.append(Input(shape=(X_train_num.shape[1],),sparse=False))
x = Dense(units=20, activation='relu')(all_inputs[-1])
all_branch_outputs.append( Dense(units=10,activation='relu')(x) )

# merge the branches together
final_branch = concatenate(all_branch_outputs)
final_branch = Dense(units=1,activation='sigmoid')(final_branch)

model = Model(inputs=all_inputs, outputs=final_branch)

model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['accuracy'])

model.fit(X_ints_train + [X_train_num],
        y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
52976/52976 [==============================] - 22s - loss: 0.1468 - acc: 0.8285    
Epoch 2/10
52976/52976 [==============================] - 21s - loss: 0.1371 - acc: 0.8285    
Epoch 3/10
52976/52976 [==============================] - 21s - loss: 0.1353 - acc: 0.8285    
Epoch 4/10
52976/52976 [==============================] - -611s - loss: 0.1339 - acc: 0.8285   
Epoch 5/10
52976/52976 [==============================] - 21s - loss: 0.1330 - acc: 0.8290    
Epoch 6/10
52976/52976 [==============================] - 21s - loss: 0.1322 - acc: 0.8305    
Epoch 7/10
52976/52976 [==============================] - 21s - loss: 0.1317 - acc: 0.8321    
Epoch 8/10
52976/52976 [==============================] - 21s - loss: 0.1314 - acc: 0.8336    
Epoch 9/10
52976/52976 [==============================] - 21s - loss: 0.1312 - acc: 0.8341    
Epoch 10/10
52976/52976 [==============================] - 21s - loss: 0.1310 - acc: 0.8343    


In [10]:
X_test_num =  df_test[numeric_headers].values
yhat = np.round(model.predict(X_ints_test + [X_test_num]))
print(mt.confusion_matrix(y_test,yhat),mt.accuracy_score(y_test,yhat))

[[10887    86]
 [ 2092   180]] 0.835560588901


In [11]:
cross_columns = [['gender','race'],
                 ['age', 'diag_1'],
                ['gender', 'diag_1']]

In [11]:
cross_columns = [['gender','race'],
                 ['age', 'diag_1'],
                ['gender', 'diag_1']]

# we need to create separate sequential models for each embedding
embed_branches = []
X_ints_train = []
X_ints_test = []
all_inputs = []
all_branch_outputs = []

for cols in cross_columns:
    # encode as ints for the embedding
    enc = LabelEncoder()
    
    # create crossed labels
    X_crossed_train = df_train[cols].apply(lambda x: '_'.join(x), axis=1)
    X_crossed_test = df_test[cols].apply(lambda x: '_'.join(x), axis=1)
    
    enc.fit(np.hstack((X_crossed_train.values,  X_crossed_test.values)))
    X_crossed_train = enc.transform(X_crossed_train)
    X_crossed_test = enc.transform(X_crossed_test)
    print(X_crossed_train)
    X_ints_train.append( X_crossed_train )
    X_ints_test.append( X_crossed_test )
    
    # get the number of categories
    N = max(X_ints_train[-1]+1) # same as the max(df_train[col])
    
    # create embedding branch from the number of categories
    inputs = Input(shape=(1,),dtype='int32')
    all_inputs.append(inputs)
    x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
    x = Flatten()(x)
    all_branch_outputs.append(x)
    


[9 9 3 ..., 3 9 3]
[1734 1955 1326 ..., 2230 2417 1091]
[ 819 1067  305 ...,  224 1033   17]


In [12]:
%%time
import tensorflow as tf
# merge the branches together
final_branch = concatenate(all_branch_outputs)
final_branch = Dense(units=1,activation='sigmoid')(final_branch)

with tf.device('/gpu:0'):
    model = Model(inputs=all_inputs, outputs=final_branch)

    model.compile(optimizer='sgd',
                  loss='mean_squared_error',
                  metrics=['accuracy'])

    model.fit(X_ints_train,
            y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
52976/52976 [==============================] - 7s - loss: 0.1699 - acc: 0.8255     
Epoch 2/10
52976/52976 [==============================] - 6s - loss: 0.1439 - acc: 0.8285     
Epoch 3/10
52976/52976 [==============================] - 6s - loss: 0.1423 - acc: 0.8285     
Epoch 4/10
52976/52976 [==============================] - 6s - loss: 0.1420 - acc: 0.8285     
Epoch 5/10
52976/52976 [==============================] - 6s - loss: 0.1419 - acc: 0.8285     
Epoch 6/10
52976/52976 [==============================] - 6s - loss: 0.1419 - acc: 0.8285     
Epoch 7/10
52976/52976 [==============================] - 6s - loss: 0.1418 - acc: 0.8285     
Epoch 8/10
52976/52976 [==============================] - ETA: 0s - loss: 0.1416 - acc: 0.828 - 6s - loss: 0.1418 - acc: 0.8285     
Epoch 9/10
52976/52976 [==============================] - 7s - loss: 0.1418 - acc: 0.8285     
Epoch 10/10
52976/52976 [==============================] - 6s - loss: 0.1417 - acc: 0.8285     
Wall time: 

In [13]:
yhat = np.round(model.predict(X_ints_test))
print(mt.confusion_matrix(y_test,yhat),mt.accuracy_score(y_test,yhat))

[[10973     0]
 [ 2272     0]] 0.828463571159


In [12]:
# we need to create separate sequential models for each embedding
import tensorflow as tf
with tf.device('/GPU:0'):
    embed_branches = []
    X_ints_train = []
    X_ints_test = []
    all_inputs = []
    all_branch_outputs = []

    for cols in cross_columns:
        # encode crossed columns as ints for the embedding
        enc = LabelEncoder()

        # create crossed labels
        # needs to be commented better, Eric!
        X_crossed_train = df_train[cols].apply(lambda x: '_'.join(x), axis=1)
        X_crossed_test = df_test[cols].apply(lambda x: '_'.join(x), axis=1)

        enc.fit(np.hstack((X_crossed_train.values,  X_crossed_test.values)))
        X_crossed_train = enc.transform(X_crossed_train)
        X_crossed_test = enc.transform(X_crossed_test)
        X_ints_train.append( X_crossed_train )
        X_ints_test.append( X_crossed_test )

        # get the number of categories
        N = max(X_ints_train[-1]+1) # same as the max(df_train[col])

        # create embedding branch from the number of categories
        inputs = Input(shape=(1,),dtype='int32', name = '_'.join(cols))
        all_inputs.append(inputs)
        x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
        x = Flatten()(x)
        all_branch_outputs.append(x)

    # merge the branches together
    wide_branch = concatenate(all_branch_outputs)

    # reset this input branch
    all_branch_outputs = []
    # add in the embeddings
    for col in categorical_headers_ints:
        # encode as ints for the embedding
        X_ints_train.append( df_train[col].values )
        X_ints_test.append( df_test[col].values )

        # get the number of categories
        N = max(X_ints_train[-1]+1) # same as the max(df_train[col])

        # create embedding branch from the number of categories
        inputs = Input(shape=(1,),dtype='int32', name=col)
        all_inputs.append(inputs)
        x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
        x = Flatten()(x)
        all_branch_outputs.append(x)

    # also get a dense branch of the numeric features
    all_inputs.append(Input(shape=(X_train_num.shape[1],),sparse=False,name='numeric_data'))
    x = Dense(units=20, activation='relu')(all_inputs[-1])
    all_branch_outputs.append( x )

    # merge the branches together
    deep_branch = concatenate(all_branch_outputs)
    deep_branch = Dense(units=50,activation='relu')(deep_branch)
    deep_branch = Dense(units=10,activation='relu')(deep_branch)

    final_branch = concatenate([wide_branch, deep_branch])
    final_branch = Dense(units=1,activation='sigmoid')(final_branch)

    model = Model(inputs=all_inputs, outputs=final_branch)

    model.compile(optimizer='adagrad',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model.fit(X_ints_train+ [X_train_num],
            y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
52976/52976 [==============================] - 30s - loss: 0.4229 - acc: 0.8343    
Epoch 2/10
52976/52976 [==============================] - 28s - loss: 0.4104 - acc: 0.8370    
Epoch 3/10
52976/52976 [==============================] - 28s - loss: 0.4048 - acc: 0.8377    
Epoch 4/10
52976/52976 [==============================] - 28s - loss: 0.4008 - acc: 0.8386    
Epoch 5/10
52976/52976 [==============================] - 29s - loss: 0.3981 - acc: 0.8387    
Epoch 6/10
52976/52976 [==============================] - 29s - loss: 0.3960 - acc: 0.8392    
Epoch 7/10
52976/52976 [==============================] - 29s - loss: 0.3943 - acc: 0.8400    
Epoch 8/10
52976/52976 [==============================] - 28s - loss: 0.3930 - acc: 0.8405    
Epoch 9/10
52976/52976 [==============================] - 28s - loss: 0.3919 - acc: 0.8410    
Epoch 10/10
52976/52976 [==============================] - 28s - loss: 0.3910 - acc: 0.8413    


In [13]:
yhat = np.round(model.predict(X_ints_test + [X_test_num]))
print(mt.confusion_matrix(y_test,yhat),mt.accuracy_score(y_test,yhat))

[[10770   203]
 [ 2030   242]] 0.83140807852


In [14]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.